# Create Simple ML Model

On this notebook we will create a base model using a simple machine learning algorithm and training it without the spatial data. This will be our baseline model to compare with the more complex models that will be created later.

The data we will use contains the following information:
- Porcentage of votes for each important party for each election held in Catalonia.
- The socio-economic index for each censal section in Catalonia.
- The age groups for each censal section in Catalonia.
- The proportion of born abroad for each censal section in Catalonia.

## Load data

First, we will load the libraries.

In [48]:
import pandas as pd
import geopandas as gpd
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import logging
import pysal as ps
import contextily
from splot.esda import plot_moran
from pysal.viz import splot
from unidecode import unidecode
from pysal.explore import esda
from pysal.lib import weights
from numpy.random import seed
from sklearn.model_selection import train_test_split
from typing import Tuple, List

pp = pprint.PrettyPrinter(indent=2)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

Now, we load the the dataset.

In [7]:
# data = gpd.read_file("../../data/output/merged_data.geojson")
# data.head()

As we won't use the geographical data, we will drop the columns that contain it.

In [8]:
# df = data.drop(columns=["geometry"])

In [9]:
# df.to_csv("../../data/output/only_votes.csv", index=False)

In [16]:
df = pd.read_csv("../../data/output/only_votes.csv", dtype=str)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5083 entries, 0 to 5082
Columns: 1392 entries, MUNICIPI to vots_valids_percentage_999999999_M20191
dtypes: object(1392)
memory usage: 54.0+ MB


## Analyze data

Each census section belong to a provice, but we don't have this information as a variable in the dataset. We will create a new variable that indicates the province of each census section based on the first two characters of the `MUNICIPI` code.

This new variable will be used to split the dataset into training and testing sets.

In [18]:
# create the province column based on the first two characters of the `MUNICIPI` column
df["province"] = df["MUNICIPI"].str[:2]

# show the proportion of each province
df["province"].value_counts(normalize=True)

province
08    0.712178
43    0.106630
17    0.104859
25    0.076333
Name: proportion, dtype: float64

The majority (71,1%) of the census sections are in the province of Barcelona, followed by Tarragona and Girona that have similar number of census sections. Lleida is the province with the least number of census sections. This could be a problem when splitting the dataset into training and testing sets, as the model could be biased towards the province of Barcelona. Also, the provinces of Tarragona, Girona and Lleida have way less census sections than Barcelona, so the model could have problems to generalize to these provinces.

We have some possible ways to tackle this problem:
- Use stratified sampling when splitting the dataset into training and testing sets. This way, the proportion of census sections from each province will be the same in the training and testing sets.
- Remove census sections from Barcelona to end up with similar proportions of census sections from each province. This way, the model will have to generalize to all provinces.
- Create a model for each province.

The last option doesn't seem correct. This would assume that the relationship between the variables and the target variable is different for each province, which is unlikely. The first option seems the best, as it will allow the model to generalize to all provinces.

## Prepare data

We want to convert this data into a format that can be used by a machine learning algorithm. The objective is to predict the percentage of votes for each party based on the socio-economic index, the age groups, the proportion of born abroad and the past percentatge of party's votes.

We have multiple variables expressing different ways to calculate the percentage of votes for each party. We will only use the `cens_electoral_percentage_*` columns.

In [19]:
# Drop `df` columns `vots_*`, `votants_percentage_*` and `vots_valids_percentage_*`
df_filtered = df.loc[:, ~df.columns.str.contains("vots_")]
df_filtered = df_filtered.loc[
    :, ~df_filtered.columns.str.contains("votants_percentage_")
]
df_filtered = df_filtered.loc[
    :, ~df_filtered.columns.str.contains("vots_valids_percentage_")
]

We set "MUNDISSEC" as the index of the dataset. And then we drop the columns that we won't use.

In [20]:
# Set "MUNDISSEC" as index
df_filtered = df_filtered.set_index("MUNDISSEC")

# Remove census section identifier columns
strarify_col = df["province"]
df_filtered = df_filtered.drop(columns=["MUNICIPI", "DISTRICTE", "SECCIO", "province"])

We will split the data into `X` and `y` where `X` contains the features and `y` contains the target variable.

We have multiple parties to predict, so we will use a multi-target regression model. Therefore, `y` will be a matrix with the percentage of votes for each party on the last election (2021).

In [60]:
import re

def generate_train_test_data(df: pd.DataFrame, target_year: int, validation_year: int) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Generates training (X_train, y_train) and testing (X_test, y_test) datasets for machine learning.

    Args:
        df (pd.DataFrame): The input dataframe containing census data, including temporal and non-temporal features.
        target_year (int): The year to be used as the prediction target for the training set.
        validation_year (int): The year used for the testing/validation set.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]: A tuple containing:
            - X_train (pd.DataFrame): The training features dataframe with temporal and non-temporal data.
            - y_train (pd.DataFrame): The target variable for the training set (target year).
            - X_test (pd.DataFrame): The testing features dataframe including X_train columns plus the target variable.
            - y_test (pd.DataFrame): The target variable for the testing set (validation year).

    Raises:
        ValueError: If the target or validation year is not found in the dataframe, or if there are no training years before the target year.
    """
    # Define a generic regex pattern to match the temporal column names, including any variable prefix
    pattern = r".*_\d+_[A-Z](\d{4})\d"
    
    # Extract all years available in the temporal columns
    all_years: List[int] = sorted(set(
        int(match.group(1)) for col in df.columns if (match := re.match(pattern, col))
    ))

    # Ensure the target and validation years are valid
    if target_year not in all_years or validation_year not in all_years:
        raise ValueError(f"Target year {target_year} or validation year {validation_year} not found in columns.")

    # Ensure target year is less than the validation year
    if target_year >= validation_year:
        raise ValueError("Target year must be less than the validation year.")
    
    # Identify the years before the target year for the feature set
    training_years: List[int] = [year for year in all_years if year < target_year]

    if not training_years:
        raise ValueError("No training years available prior to the target year.")

    # Separate temporal and non-temporal columns
    training_columns_temporal: List[str] = [
        col for col in df.columns if any(re.match(rf".*_\d+_[A-Z]{year}\d", col) for year in training_years)
    ]
    target_column: List[str] = [col for col in df.columns if re.match(rf".*_\d+_[A-Z]{target_year}\d", col)]
    validation_column: List[str] = [col for col in df.columns if re.match(rf".*_\d+_[A-Z]{validation_year}\d", col)]
    non_temporal_columns: List[str] = [
        col for col in df.columns if not re.match(pattern, col)
    ]

    if not target_column or not validation_column:
        raise ValueError("Missing target or validation columns.")

    # Combine temporal and non-temporal columns for training
    training_columns: List[str] = training_columns_temporal + non_temporal_columns

    # Extract X_train (features), y_train (target year labels)
    X_train: pd.DataFrame = df[training_columns]
    y_train: pd.DataFrame = df[target_column]

    # Prepare X_test using the same features as X_train, plus the target year labels as features
    test_columns: List[str] = training_columns + target_column
    X_test: pd.DataFrame = df[test_columns]
    y_test: pd.DataFrame = df[validation_column]

    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = generate_train_test_data(
    df_filtered, 2019, 2021
)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (5083, 296)
X_test shape: (5083, 338)
y_train shape: (5083, 42)
y_test shape: (5083, 9)


We have 5083 censal sections and 9 parties. Now we will split the data into training and test sets.

- The training set will be used to train the model and the test set will be used to evaluate the model. It will cointain 80% of the data.
- The test set will contain the remaining 20% of the data.

In [52]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y_target, test_size=0.35, stratify=strarify_col, random_state=42
# )

# print(f"X_train shape: {X_train.shape}")
# print(f"X_test shape: {X_test.shape}")
# print(f"y_train shape: {y_train.shape}")
# print(f"y_test shape: {y_test.shape}")

NameError: name 'y_target' is not defined

In [24]:
print(
    f"X_train proportion of provinces: {X_train.index.str[:2].value_counts(normalize=True)}"
)
print(
    f"X_test proportion of provinces: {X_test.index.str[:2].value_counts(normalize=True)}"
)
print(
    f"y_train proportion of provinces: {y_train.index.str[:2].value_counts(normalize=True)}"
)
print(
    f"y_test proportion of provinces: {y_test.index.str[:2].value_counts(normalize=True)}"
)

X_train proprotion of provinces: MUNDISSEC
08    0.712080
43    0.106570
17    0.105056
25    0.076294
Name: proportion, dtype: float64
X_test proprotion of provinces: MUNDISSEC
08    0.712360
43    0.106742
17    0.104494
25    0.076404
Name: proportion, dtype: float64
y_train proprotion of provinces: MUNDISSEC
08    0.712080
43    0.106570
17    0.105056
25    0.076294
Name: proportion, dtype: float64
y_test proprotion of provinces: MUNDISSEC
08    0.712360
43    0.106742
17    0.104494
25    0.076404
Name: proportion, dtype: float64


## Multi-Target Learning in XGBoost 2.0

### Model definition

In [26]:
import xgboost as xgb

# Convert to float type
X_train = X_train.astype(float)
X_test = X_test.astype(float)
y_train = y_train.astype(float)
y_test = y_test.astype(float)

# Create DMatrix objects for training and testing
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [27]:
# Parameters specific to the multi-target regression task
params = {
    "objective": "reg:squarederror",  # Squared error regression objective
    "max_depth": 6,                   # Depth of each tree
    "eta": 0.1,                       # Learning rate
    "num_parallel_tree": 1            # Number of parallel trees (boosting round)
}

### Model training

In [28]:
# Train the model
num_boost_round = 200
model = xgb.train(params, dtrain, num_boost_round)

### Model evaluation

In [29]:
# Predict the targets for the test data
y_pred = model.predict(dtest)

In [30]:
y_pred.shape

(1780, 9)

In [31]:
# Convert the predictions to a DataFrame
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns, index=y_test.index)

y_pred_df.head()

,cens_electoral_percentage_6_A20211,cens_electoral_percentage_10_A20211,cens_electoral_percentage_86_A20211,cens_electoral_percentage_301_A20211,cens_electoral_percentage_693_A20211,cens_electoral_percentage_1031_A20211,cens_electoral_percentage_1099_A20211,cens_electoral_percentage_2019838_A20211,cens_electoral_percentage_999999999_A20211
MUNDISSEC,,,,,,,,,
08033303005,9.084556,14.076160,1.605601,4.005919,5.085562,10.004305,3.173017,3.299025,0.243505
08089803004,17.172112,6.528187,1.308153,3.016288,4.198141,2.043963,3.417904,1.221474,0.116289
08056905005,13.027328,10.488903,3.240808,4.335413,5.210976,7.141871,3.801031,2.443593,0.239999
08248201001,10.478132,13.655977,1.249182,3.102514,3.496953,10.457832,2.913667,3.443631,0.269708
08172703009,10.348579,12.475167,2.297074,3.568926,5.466607,13.781796,3.773765,3.522207,0.399626


In [32]:
y_pred_df.head(2)

,cens_electoral_percentage_6_A20211,cens_electoral_percentage_10_A20211,cens_electoral_percentage_86_A20211,cens_electoral_percentage_301_A20211,cens_electoral_percentage_693_A20211,cens_electoral_percentage_1031_A20211,cens_electoral_percentage_1099_A20211,cens_electoral_percentage_2019838_A20211,cens_electoral_percentage_999999999_A20211
MUNDISSEC,,,,,,,,,
08033303005,9.084556,14.076160,1.605601,4.005919,5.085562,10.004305,3.173017,3.299025,0.243505
08089803004,17.172112,6.528187,1.308153,3.016288,4.198141,2.043963,3.417904,1.221474,0.116289


In [33]:
y_test.head()

,cens_electoral_percentage_6_A20211,cens_electoral_percentage_10_A20211,cens_electoral_percentage_86_A20211,cens_electoral_percentage_301_A20211,cens_electoral_percentage_693_A20211,cens_electoral_percentage_1031_A20211,cens_electoral_percentage_1099_A20211,cens_electoral_percentage_2019838_A20211,cens_electoral_percentage_999999999_A20211
MUNDISSEC,,,,,,,,,
08033303005,10.523256,14.825581,1.918605,3.662791,4.941860,10.988372,2.732558,3.837209,0.207641
08089803004,15.559294,5.466779,1.261564,3.195963,4.457527,2.943650,3.784693,1.177460,0.108134
08056905005,14.361350,13.567174,4.169424,6.882859,5.757776,7.279947,5.228326,2.448709,0.236362
08248201001,9.962342,12.906539,1.300924,2.362205,3.252311,11.502910,3.423485,3.731599,0.190737
08172703009,10.521500,13.174748,2.104300,4.208600,6.404392,13.998170,4.208600,3.110704,0.392106


In [34]:
import re

def parse_column_name(column_name: str, variable_name: str = "cens_electoral_percentage"):
    """
    Parse the column name to extract party code, election type, year, and repetition.

    Args:
        column_name (str): Column name formatted as `VARIABLE_PARTYCODE_ELECTIONSCODE`.
        variable_name (str): The prefix used in the column name.

    Returns:
        dict: A dictionary containing parsed components: party_code, election_type, election_year, election_repetition.
    """
    # Dynamic regex pattern using the provided variable name
    pattern = rf"{re.escape(variable_name)}_(\d+)_([A-Z])(\d{{4}})(\d)"

    # Use regex to find all elements
    match = re.match(pattern, column_name)

    if match:
        return {
            'party_code': match.group(1),
            'election_type': match.group(2),
            'election_year': match.group(3),
            'election_repetition': match.group(4)
        }
    else:
        raise ValueError(f"Invalid column name format: {column_name}")

In [35]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Ensure y_test and y_pred are NumPy arrays
y_test_np = y_test.values if isinstance(y_test, pd.DataFrame) else np.array(y_test)
y_pred_np = y_pred.values if isinstance(y_pred, pd.DataFrame) else np.array(y_pred)

# Retrieve the column names from the DataFrame (y_test)
columns = y_test.columns

# Initialize lists to hold metrics for each target
mae = []
mse = []
rmse = []
r2 = []

# Calculate metrics for each target column
for i in range(y_test_np.shape[1]):
    mae.append(mean_absolute_error(y_test_np[:, i], y_pred_np[:, i]))
    mse.append(mean_squared_error(y_test_np[:, i], y_pred_np[:, i]))
    rmse.append(np.sqrt(mse[i]))
    r2.append(r2_score(y_test_np[:, i], y_pred_np[:, i]))

    # Extract the party code from the column name using the parse function
    parsed_info = parse_column_name(columns[i])

    # Display results for each target
    print(f"Target {i+1} (Party Code: {parsed_info['party_code']}):")
    print(f" - MAE: {mae[i]:.4f}")
    print(f" - MSE: {mse[i]:.4f}")
    print(f" - RMSE: {rmse[i]:.4f}")
    print(f" - R²: {r2[i]:.4f}")
    print()


Target 1 (Party Code: 6):
 - MAE: 1.0812
 - MSE: 2.1651
 - RMSE: 1.4714
 - R²: 0.9027

Target 2 (Party Code: 10):
 - MAE: 1.0766
 - MSE: 2.6543
 - RMSE: 1.6292
 - R²: 0.8735

Target 3 (Party Code: 86):
 - MAE: 0.4701
 - MSE: 0.5247
 - RMSE: 0.7243
 - R²: 0.8087

Target 4 (Party Code: 301):
 - MAE: 0.5242
 - MSE: 0.4683
 - RMSE: 0.6843
 - R²: 0.7889

Target 5 (Party Code: 693):
 - MAE: 0.6199
 - MSE: 0.6666
 - RMSE: 0.8165
 - R²: 0.8218

Target 6 (Party Code: 1031):
 - MAE: 1.0166
 - MSE: 3.4576
 - RMSE: 1.8595
 - R²: 0.9438

Target 7 (Party Code: 1099):
 - MAE: 0.5873
 - MSE: 0.6205
 - RMSE: 0.7877
 - R²: 0.8004

Target 8 (Party Code: 2019838):
 - MAE: 0.6071
 - MSE: 0.8589
 - RMSE: 0.9267
 - R²: 0.8641

Target 9 (Party Code: 999999999):
 - MAE: 0.0810
 - MSE: 0.0193
 - RMSE: 0.1390
 - R²: 0.6074



In [36]:
X.columns

Index(['cens_electoral_percentage_4_A19801',
       'cens_electoral_percentage_4_A19841',
       'cens_electoral_percentage_4_E19871',
       'cens_electoral_percentage_4_E19891',
       'cens_electoral_percentage_4_E20041',
       'cens_electoral_percentage_4_G19771',
       'cens_electoral_percentage_4_G19791',
       'cens_electoral_percentage_4_G19821',
       'cens_electoral_percentage_4_G19861',
       'cens_electoral_percentage_4_G19891',
       ...
       'cens_electoral_percentage_999999999_M19831',
       'cens_electoral_percentage_999999999_M19871',
       'cens_electoral_percentage_999999999_M19911',
       'cens_electoral_percentage_999999999_M19951',
       'cens_electoral_percentage_999999999_M19991',
       'cens_electoral_percentage_999999999_M20031',
       'cens_electoral_percentage_999999999_M20071',
       'cens_electoral_percentage_999999999_M20111',
       'cens_electoral_percentage_999999999_M20151',
       'cens_electoral_percentage_999999999_M20191'],
      dt

## PyCaret Experimentation

PyCaret is a low-code machine learning library that allows you to go from preparing your data to deploying your model within minutes in your choice of notebook environment.

In [37]:
target_list = y_target.columns.tolist()

In [41]:
df_filtered = df_filtered.astype(float)

In [42]:
from pycaret.regression import RegressionExperiment

s = RegressionExperiment()
s.setup(df_filtered, target = target_list[0], session_id = 123)

,Description,Value
0,Session id,123
1,Target,cens_electoral_percentage_6_A20211
2,Target type,Regression
3,Original data shape,"(5083, 347)"
4,Transformed data shape,"(5083, 347)"
5,Transformed train set shape,"(3558, 347)"
6,Transformed test set shape,"(1525, 347)"
7,Numeric features,346
8,Rows with missing values,1.8%
9,Preprocess,True


In [43]:
# compare models
best = s.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.1165,2.2370,1.4944,0.9003,0.1517,0.1182,0.5150
et,Extra Trees Regressor,1.1369,2.3076,1.5178,0.8973,0.1477,0.1193,2.3250
gbr,Gradient Boosting Regressor,1.1389,2.3665,1.5361,0.8947,0.1545,0.1213,4.3000
br,Bayesian Ridge,1.1487,2.5153,1.5801,0.8874,0.1595,0.1226,0.0620
rf,Random Forest Regressor,1.1869,2.5264,1.5875,0.8873,0.1530,0.1243,7.4330
ridge,Ridge Regression,1.1776,2.5593,1.5956,0.8855,0.1648,0.1266,0.0440
lr,Linear Regression,1.1815,2.5775,1.6013,0.8847,0.1660,0.1271,0.5500
xgboost,Extreme Gradient Boosting,1.1984,2.6025,1.6112,0.8841,0.1608,0.1249,1.2390
huber,Huber Regressor,1.1325,2.7668,1.6491,0.8760,0.1750,0.1186,0.3010
omp,Orthogonal Matching Pursuit,1.2285,2.9056,1.6935,0.8701,0.1656,0.1290,0.0410


In [44]:
print(best)

LGBMRegressor(n_jobs=-1, random_state=123)


In [45]:
# analyze the best model
s.evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [46]:
for target in target_list:
    s = RegressionExperiment()
    s.setup(df_filtered, target = target, session_id = 123)
    best = s.compare_models()
    print(best)

,Description,Value
0,Session id,123
1,Target,cens_electoral_percentage_6_A20211
2,Target type,Regression
3,Original data shape,"(5083, 347)"
4,Transformed data shape,"(5083, 347)"
5,Transformed train set shape,"(3558, 347)"
6,Transformed test set shape,"(1525, 347)"
7,Numeric features,346
8,Rows with missing values,1.8%
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.1165,2.2370,1.4944,0.9003,0.1517,0.1182,0.5210
et,Extra Trees Regressor,1.1369,2.3076,1.5178,0.8973,0.1477,0.1193,2.3000
gbr,Gradient Boosting Regressor,1.1389,2.3665,1.5361,0.8947,0.1545,0.1213,4.2560
br,Bayesian Ridge,1.1487,2.5153,1.5801,0.8874,0.1595,0.1226,0.0600
rf,Random Forest Regressor,1.1869,2.5264,1.5875,0.8873,0.1530,0.1243,6.5990
ridge,Ridge Regression,1.1776,2.5593,1.5956,0.8855,0.1648,0.1266,0.0400
lr,Linear Regression,1.1815,2.5775,1.6013,0.8847,0.1660,0.1271,0.5080
xgboost,Extreme Gradient Boosting,1.1984,2.6025,1.6112,0.8841,0.1608,0.1249,1.2230
huber,Huber Regressor,1.1325,2.7668,1.6491,0.8760,0.1750,0.1186,0.3080
omp,Orthogonal Matching Pursuit,1.2285,2.9056,1.6935,0.8701,0.1656,0.1290,0.0400


LGBMRegressor(n_jobs=-1, random_state=123)


,Description,Value
0,Session id,123
1,Target,cens_electoral_percentage_10_A20211
2,Target type,Regression
3,Original data shape,"(5083, 347)"
4,Transformed data shape,"(5083, 347)"
5,Transformed train set shape,"(3558, 347)"
6,Transformed test set shape,"(1525, 347)"
7,Numeric features,346
8,Rows with missing values,1.8%
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.0371,2.3111,1.5153,0.8861,0.1204,0.1029,2.3470
gbr,Gradient Boosting Regressor,1.0486,2.3131,1.5168,0.8858,0.1242,0.1068,4.5080
lightgbm,Light Gradient Boosting Machine,1.0502,2.3317,1.5211,0.8850,0.1222,0.1040,0.6170
rf,Random Forest Regressor,1.0767,2.5218,1.5811,0.8757,0.1257,0.1071,6.8000
xgboost,Extreme Gradient Boosting,1.1088,2.5959,1.6057,0.8723,0.1291,0.1102,1.3410
omp,Orthogonal Matching Pursuit,1.1110,2.6844,1.6289,0.8677,0.1324,0.1095,0.0400
en,Elastic Net,1.1156,2.7152,1.6425,0.8661,0.1421,0.1122,0.0420
huber,Huber Regressor,1.0751,2.8191,1.6619,0.8614,0.1581,0.1047,0.3290
br,Bayesian Ridge,1.0963,2.8400,1.6532,0.8608,0.1311,0.1089,0.0550
llar,Lasso Least Angle Regression,1.1658,2.9133,1.7025,0.8563,0.1526,0.1195,0.0390


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.6042,0.8700,0.9280,0.8755,0.1817,0.2209,2.2040
rf,Random Forest Regressor,0.6130,0.9583,0.9713,0.8627,0.1872,0.2273,13.5580
gbr,Gradient Boosting Regressor,0.6185,0.9616,0.9744,0.8626,0.1888,0.2322,3.5930
lightgbm,Light Gradient Boosting Machine,0.6142,0.9633,0.9761,0.8625,0.1880,0.2257,0.7530
omp,Orthogonal Matching Pursuit,0.6388,0.9894,0.9851,0.8585,0.1990,0.2303,0.0900
br,Bayesian Ridge,0.6562,0.9989,0.9918,0.8575,0.2056,0.2420,0.1370
xgboost,Extreme Gradient Boosting,0.6370,1.0120,1.0014,0.8555,0.1946,0.2345,1.3930
huber,Huber Regressor,0.6571,1.0247,1.0052,0.8535,0.2099,0.2361,0.5900
en,Elastic Net,0.6621,1.0684,1.0290,0.8472,0.1987,0.2374,0.0940
ridge,Ridge Regression,0.6856,1.0760,1.0293,0.8465,0.2182,0.2589,0.0950


ExtraTreesRegressor(n_jobs=-1, random_state=123)


,Description,Value
0,Session id,123
1,Target,cens_electoral_percentage_999999999_A20211
2,Target type,Regression
3,Original data shape,"(5083, 347)"
4,Transformed data shape,"(5083, 347)"
5,Transformed train set shape,"(3558, 347)"
6,Transformed test set shape,"(1525, 347)"
7,Numeric features,346
8,Rows with missing values,1.8%
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0857,0.0231,0.1502,0.5831,0.0928,0.3628,0.6140
et,Extra Trees Regressor,0.0864,0.0241,0.1530,0.5684,0.0941,0.3728,2.6960
xgboost,Extreme Gradient Boosting,0.0897,0.0243,0.1546,0.5592,0.0964,0.3777,1.4600
rf,Random Forest Regressor,0.0872,0.0245,0.1550,0.5588,0.0955,0.3787,7.9510
br,Bayesian Ridge,0.0951,0.0246,0.1554,0.5546,0.0986,0.4250,0.0510
gbr,Gradient Boosting Regressor,0.0883,0.0257,0.1583,0.5398,0.0966,0.3812,4.0830
omp,Orthogonal Matching Pursuit,0.0985,0.0256,0.1586,0.5361,0.1005,0.4447,0.0280
ridge,Ridge Regression,0.1002,0.0259,0.1595,0.5301,0.1025,0.4580,0.0290
lr,Linear Regression,0.1005,0.0260,0.1600,0.5274,0.1028,0.4599,0.0370
knn,K Neighbors Regressor,0.0943,0.0270,0.1618,0.5174,0.1004,0.4024,0.0390


LGBMRegressor(n_jobs=-1, random_state=123)
